**<h2>Tokenising 100 Files in the Corpus sentence-wise first and then word-wise.<h2/>**
<h3>All the tokenising into sentences and word-tokens is done in the below cell.Furthermore for spelling prediction I have made spelling n-grams as well in the below-cell.<br/>
After that I have plotted the top-10 and bottom-10 occurring unigrams in a table.</h3>


In [605]:
import re
import os
import glob
import ast
from ipy_table import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *
import math



regex_str = [
            r"[\w]+\-[\w]+\-[\w]+\-[\w]+\-[\w]+",          # HYPHEN
            r"\w+(?:-\w+)+",
            r"[\w]+\'[\w]+",          #APOSTROPHE  
            r"\b[A-Z][a-zA-Z\.]*[A-Z]\b\.?",        # Abbreviations
            r"[a-zA-Z]+",  #normal words
            r"[^\s\,\?\!\-\_\:\"\'\;]+", 
            r"[^\s]", #anything but space
]

sentences_str = [
        r"([A-Z][^\.!?]*[\.!?])"
]
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
sentences_re = re.compile(r'('+'|'.join(sentences_str)+')',re.VERBOSE | re.IGNORECASE)

def sentence_tokenize(s):
    return sentences_re.findall(s)
    

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    return tokens


arr = [] 
path = './../Gutenberg/txt/'

tokens_i_do_not_want = {",",'"'," ","-","!",";","_",":"}
spelling_unigrams = {}
spelling_bigrams = {}
spelling_trigrams = {}
unigrams = {}
bigrams = {}
trigrams = {}
kn_unigram_probs = {}
kn_bigram_probs = {}
lp_bigram_probs = {}
count = 0 
for filename in glob.glob(os.path.join(path, '*.txt')):
    count = count + 1
    if count <= 100:
        f1 = open(filename, 'r')
        varLine = f1.read()
        bArray = varLine.replace("\n"," ")
        bArray = sentence_tokenize(bArray)

        for sentences in bArray:
            l = preprocess(sentences[0])

            for word in l:
                word = word.lower()
                if word not in tokens_i_do_not_want:
                    if word not in unigrams:
                        unigrams[word] = 0
                    unigrams[word] += 1
                    for index,elements in enumerate(word):
                        """ bigram  frequency """
                        if index < len(word) - 1:
                            w1 = word[index].lower()
                            w2 = word[index+1].lower()
                            spelling_gram = w1 + w2
                            if spelling_gram in spelling_bigrams:
                                spelling_bigrams[ spelling_gram ] = spelling_bigrams[ spelling_gram ] + 1
                            else:
                                spelling_bigrams[ spelling_gram ] = 1
                           
                            if word[index] not in spelling_unigrams:
                                spelling_unigrams[word[index]] = 0
                            spelling_unigrams[word[index]] += 1
                        else:
                            if word[index] not in spelling_unigrams:
                                spelling_unigrams[word[index]] = 0
                            spelling_unigrams[word[index]] += 1
                        
                        """ trigram frequency"""
                        
                        if index < len(word) - 2:
                            w1 = word[index].lower()
                            w2 = word[index+1].lower()
                            w3 = word[index+2].lower()
                            spelling_gram = w1 + w2 + w3
                            if spelling_gram in spelling_trigrams:
                                spelling_trigrams[ spelling_gram ] = spelling_trigrams[ spelling_gram ] + 1
                            else:
                                spelling_trigrams[ spelling_gram ] = 1
                    
                    
            for index, word in enumerate(l):
                if index < len(l) - 1:
                    # we only look at indices up to the
                    # next-to-last word, as this is
                    # the last one at which a bigram starts
                    w1 = l[index].lower()
                    w2 = l[index + 1].lower()
                    # bigram is a tuple,
                    # like a list, but fixed.
                    # Tuples can be keys in a dictionary
                    if w1 not in tokens_i_do_not_want and w2 not in tokens_i_do_not_want:
                        bigram = w1 + " " + w2
                        if bigram in bigrams:
                            bigrams[ bigram ] = bigrams[ bigram ] + 1
                        else:
                            bigrams[ bigram ] = 1
                else:
                    break


    f1.close()



sorted_tokens = sorted(unigrams.items(), key = lambda x: x[1], reverse=True)


trace = go.Table(
    header=dict(values=['Top-5 Unigrams', 'Bottom-5 Unigrams']),
    cells=dict(values=[sorted_tokens[0:5],
                       sorted_tokens[-5:]]))

data = [trace] 
iplot(data, filename = 'basic_table')


<h3> Plot of Unigrams vs log of their frequency </h3>

In [ ]:
Arr1 = [log(z,10) for m,z in sorted_tokens]
boom1 = [m for m,z in sorted_tokens] 
iplot([{"x" : Arr1, "y": boom1}])


**<h2>For the Language model, I've taken a bi-gram LM and applied Knesser Ney smoothing to predict the likelihood of the all the bi-grams</h2>**

In [607]:
def kneser_ney(bigram):
    
    D = 0.75
    u_sum = sum(unigrams.values())
    """Finding knesser ney probabilities for unigrams"""
    for x in unigrams:
        kn_unigram_probs[x] = unigrams[x]/u_sum
        
    """Finding knesser ney probabilities for Bigrams"""

    arr = bigrams.keys()
    count = 0
    count1 = 0
    count2 = 0
    first_word = bigram.split(" ")[0]
    second_word = bigram.split(" ")[1]
    for x in arr:
        abc = x.split()
        if second_word == abc[1]:
            count += 1
        if first_word == abc[0]:
            count2 += 1
            
    count1 = unigrams[first_word]
    lamda = D*count2/count1


    if bigrams[bigram] == 0:
        kn_bigram_probs[bigram] = (lamda*count)/len(bigrams)
    else: 
        kn_bigram_probs[bigram] = max((bigrams[bigram] - D),0)/unigrams[first_word]
        
    return kn_bigram_probs[bigram]  
    


In [608]:
def language_model(bigrams,bigram):
# Build your language model
# For any language model you build, print a sample of the top 10 and the bottom 10 frequently occuring N-grams.
# For any language model you build, display plots of N-Gram vs Freq sorted by frequency (descending) and log-log plots of the same
    if bigram in bigrams:
        prob = kneser_ney(bigram)
    else:
        bigrams[bigram] = 0
        prob = kneser_ney(bigram)
    return prob
    
print("The possibility of the bigram :",language_model(bigrams,"who what"))

The possibility of the bigram : 6.825226256677224e-05


<br><h2> Table consisting of the Top-10 and the Bottom-10 frequently occuring bi-grams. </h2>

In [609]:
sorted_bigrams = sorted(bigrams.items(), key = lambda x: x[1], reverse=True)
trace = go.Table(
    header=dict(values=['Top-10 Bigrams', 'Bottom-10 Bigrams']),
    cells=dict(values=[sorted_bigrams[0:10],
                       sorted_bigrams[-10:]]))

data = [trace] 
iplot(data, filename = 'basic_table')



<br><h2> Graphs of the Top-100 frequently occuring bi-grams along with their n-gram vs frequency plot and log-log plot. </h2>

In [610]:
iplot([{"x" : list(zip(*sorted_bigrams[0:100]))[0], "y": list(zip(*sorted_bigrams[0:100]))[1]}])
Arr = []
count = 0
Arr = [log(z,10) for m,z in sorted_bigrams[0:100]]
boom = [m for m,z in sorted_bigrams[0:100]]     
iplot([{"x" : boom, "y": Arr}])


<br>

<br><h2>To calculate all the bigram probabilities of the bi-gram model I have used Laplacian smoothing with factor 50 or 100 whichever gives me better results</h2><br/>
<h3>This creates a list containing the smoothed bi-gram probabilities</h3>

In [611]:

def laplace_bigram_probability(bigrams):
    
    
    for bigram in bigrams:
        arr = bigram.split(" ")
        bigram = arr[0].lower() + " " + arr[1].lower()
        n_bigram = 0
        
        if bigrams[bigram] != 0:
            n_bigram = bigrams[bigram]

        first_word=arr[0]
        n_first_word=0

        if first_word in unigrams:
            n_first_word = unigrams[first_word]

        V=len(unigrams)
        
        lp_bigram_probs[bigram] =(n_bigram+0.14*1)/float(n_first_word+100) #a simple add 1 smoothing
    
    return 1

bigram_probability(bigrams)
print("I am :",lp_bigram_probs["i am"])
bigrams["who what"] = 0 #Unseen word in the dictionary
bigram_probability(bigrams)
print("who what",lp_bigram_probs["who what"]) # Some mass is alloted to this bigram

I am : 0.06435985736828799
who what 6.769825918762089e-06


<br>
<h3>Implementing Backoff (Katz 1987) on bigrams. </h3>
<br>

In [612]:
backoff_prob = {}
def backoff(bigram):
    alpha1 = 0.75
    alpha2 = 0.25
    arr = bigram.split()
    if bigrams[bigram] > 0:
        language_model(bigrams,bigram)
        backoff_prob[bigram] = kn_bigram_probs[bigram]
    elif bigrams[bigram] == 0 and unigrams[arr[0]] > 0:
        bigrams[bigram] = 0
        language_model(bigrams,bigram)
        backoff_prob[bigram] = alpha1*kn_bigram_probs[bigram]
    else:
        language_model(bigrams,bigram)
        backoff_prob[bigram] = alpha2*kn_unigram_probs[arr[0]]
    
    return backoff_prob[bigram]


print("Probability of Bigram using back-off and Kneser-Ney : ",backoff("am i"))
        

Probability of Bigram using back-off and Kneser-Ney :  0.048969704403287136


<br><h2><em>Like the language-model I break the word tokens into trigrams,bigrams and unigrams and using a simple add-one smoothing on this model, I find the likelihood of the word</em>.</h2>

In [613]:
def spell_prob(word):
    
    spelling_tokens = []
    for index,y in enumerate(word):
        spelling_tokens.append(word[index].lower())
    
    prev_spelling_token = spelling_tokens[0]+spelling_tokens[1]
    new_spelling_trigrams = []
    word_likelihood=0
    
    for index in range(2,len(spelling_tokens)):
        next_spelling_token=spelling_tokens[index]
        spelling_bigram=prev_spelling_token + next_spelling_token
        new_spelling_trigrams.append(spelling_bigram)
        prev_spelling_token=spelling_tokens[index-1] + next_spelling_token
    
    """
    finding trigram likelihood of the word
    """
    index = 0
    for spelling_trigram in new_spelling_trigrams:
        n_trigram = 0
        if spelling_trigram in spelling_trigrams:
            n_trigram = spelling_trigrams[spelling_trigram]
        
        first_bigram=spelling_trigram[index] + spelling_trigram[index+1]
        
        n_first_letter=0
        
        if first_bigram in spelling_bigrams:
            n_first_letter = spelling_bigrams[first_bigram]
        
        V=len(spelling_bigrams)
        
        trigram_likelihood=(n_trigram+1)/float(n_first_letter+100) #a simple add 1 smoothing
        word_likelihood += -math.log(trigram_likelihood)
        
    return word_likelihood

print(spell_prob("beautiful"))
def spell_checker(word,prob):
    
    if len(word) == 3:
        if prob <= 3.3:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")
    if len(word) == 4:
        if prob <= 7.1:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")
    if len(word) == 5:
        if prob <= 8.5:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")
    if len(word) == 6:
        if prob <= 10:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")
    if len(word) == 7:
        if prob <= 13.7:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")
    if len(word) == 8:
        if prob <= 16.5:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")
    if len(word) == 9:
        if prob <= 19.2:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")
    if len(word) == 10:
        if prob <= 24:
            print("Correct Spelling")
        else:
            print("Incorrect Spelling")

spell_checker("wonderful",spell_prob("wonderful"))


19.10468576450374
Correct Spelling


<br><h2><em>I find the set of all those bi-grams which has the last word of the sentence as it first word and simultaneously find the probabilities associated with them to predict the next word.</em></h2>

In [614]:
tokens_i_do_not_want.add("?")
def complete_ngram(ngram,n):
    words = ngram.split(" ")
    first_word = words[len(words)-1]
    
    arr = bigrams.keys()
    
    poss_bigrams = {}
    for x in arr:
        y = x.split(" ")
        
        if first_word == y[0]:
            poss_bigrams[x] = lp_bigram_probs[x]
    poss_bigrams = sorted(poss_bigrams.items(), key = lambda x: x[1], reverse=True)
    for x in poss_bigrams[:10]:
        var = x[0].split(" ")
        if var[1] != words[len(words)-2] and var[1] not in tokens_i_do_not_want:
            print(ngram+" " +var[1] +" : ", x[1])
       

complete_ngram("let me know",4)

let me know that :  0.10698671321889246
let me know what :  0.09437881873727087
let me know the :  0.04714770633304238
let me know how :  0.046177868295994566
let me know it :  0.03405489283289691
let me know you :  0.02775094559208612
let me know i :  0.020671127921637084
let me know not :  0.019798273688294052
let me know where :  0.019604306080884492
let me know of :  0.01727669479196974


<br><br>

<h2><em>I have used the Knesser-Ney smoothing on bi-gram model to process the likelihood of the sentence.</em></h2>
<p><em>The lesser the score for the sentence the higher is the probability of that sentence being valid.</em></p>

In [615]:
def score_grammaticality(sentence):
    words = preprocess(sentence)
    bigram_list = {}
    for index,word in enumerate(words):
        if index < len(words) - 1:
            word1 = words[index].lower()
            word2 = words[index+1].lower()
            bigram = word1 + " " + word2
            if bigram not in bigrams:
                bigrams[bigram] = 0
            bigram_list[bigram] = bigrams[bigram]
    
    prob = 0
    for x in bigram_list:
        prob += -math.log(language_model(bigrams,x))
    return prob
print("S1: apple a have red i :",score_grammaticality("apple a have red i")) 
print("S2: I have a red apple :",score_grammaticality("I have a red apple"))

S1: apple a have red i : 36.188283950778676
S2: I have a red apple : 25.39373651910472


<h3><b> Since "I have a red apple" has a lower score than "apple a have red i" we can conclude that S1 is worse than S2</b></h3>
